# IP in het lokale netwerk

```{admonition} Concepten
MAC-adres, IP-adres, bus, Ethernet, WiFi, frame, collision, LAN-routering, switch, gateway
```

## Lokale IP-communicatie

De communicatie van IP-pakketten gebeurt in het lokale netwerk op basis van de Ethernet- en WiFi-verbindingen. Een IP-pakket wordt hiervoor ingepakt in een Ethernet- of WiFi-frame, wat voorzien moet worden van de juiste MAC-adresssen.

```{figure} images/ip-in-ethernet-frame.png
:width: 600px

IP-pakket ingepakt in een Ethernet frame
```

Als een host in het lokale netwerk een IP-pakket wil versturen, bepaalt deze host eerst of de bestemming een IP-adres in het lokale netwerk is, of daarbuiten. Voor een pakket in het lokale netwerk wordt het MAC-adres van de bestemming gebruikt. Een pakket met een bestemming buiten het lokale netwerk krijgt het MAC-adres van de gateway: die stuurt het pakket verder.

*Hoe bepaalt een host of een IP-adres een adres in het lokale netwerk is?* Dit gebeurt aan de hand van het *netmasker*. Het netmasker geeft aan welke bits tot de *netwerkprefix* van het adres horen, en welke het host-adres in het netwerk vormen. Als de netwerkprefix van de bestemming verschilt van de eigen netwerkprefix van de host, dan is het pakket voor een ander netwerk bestemd.

```{admonition} Voorbeeld
Het netmasker is 255.255.255.0, en het IP-adres van de host is 192.168.3.42. Dit betekent dat de netwerkprefix 192.168.3.0 is, en het host-deel van het adres 42.


| netmasker (decimaal)         | 255.255.255.0                         |
| :---                         | :---                                  |
| IP-adres (decimaal)          | 192.168.3.42                          |
| IP-adres (binair)            | `11000000.10101000.00000011.00101010` |
| netmasker (binair)           | `11111111.11111111.11111111.00000000` |
| "and" van masker en IP-adres | `11000000.10101000.00000011.00000000` |
| netwerkadres (decimaal)      | 192.168.3.0                           |

IP-adres 192.168.3.27 valt in hetzelfde netwerk. IP-adres 192.168.7.27 valt in een ander netwerk, net als 27.63.12.13.

```

*Hoe weet een host het MAC-adres van een andere host in het lokale netwerk, op basis van het IP-adres daarvan?*
Elke host houdt daarvoor een tabel bij met lokale IP-adressen en bijbehorende MAC-adressen.
Om deze tabel te vullen voor een bepaald IP-adres, stuurt de host een verzoek naar alle andere hosts in het netwerk (*broadcast*), met daarin het gezochte IP-adres.
De host met dit gezochte IP-adres stuurt een antwoord terug met daarin het eigen MAC-adres ingevuld. (Address Resolution Protocol, ARP).

Je kunt uit het netmasker ook bepalen hoeveel hosts er maximaal in het netwerk kunnen zijn.
De 0-en aan het eind van het netmasker zijn beschikbaar voor het netwerk zelf.
*Voorbeeld.* Acht 0-en geeft 256 mogelijke adressen; daarvan zijn er 2 gereserveerd: `00000000` is het netwerkadres; `11111111` is het broadcast-adres. Bovendien heb je nog een adres nodig voor de gateway; deze heeft meestal adres `00000001` of `11111110` (decimaal 254). Blijven er 253 over voor hosts in het netwerk.

## Netwerk-prefix

Een IP-adres bestaat uit twee delen: het adres van het netwerk (de netwerk-prefix), en het adres van host in het netwerk. Bij IPv4 zijn er twee manieren om de grens tussen deze twee delen aan te geven:

* door de prefix-lengte, in de notatie `/xx`, waarin *xx* het aantal bits van de prefix aangeeft.
* door het netmasker, in decimale notatie voor de bytes, bijvoorbeeld `255.255.254.0`. Hierin geven de "1"-bits aan welk deel tot de prefix behoort.

Voor het uitrekenen van de lengte uit een byte-masker (of omgekeerd) kun je de volgende tabel gebruiken:

```{figure} images/prefix-tabel.png
:width: 300px

Verband tussen byte-masker en prefix-lengte
```

Voorbeeld: het netmasker `255.255.254.0` komt overeen met prefix-lengte 8 + 8 + 7 + 0 = 23. Het netmasker `255.255.248.0` geeft prefix-lengte 8 + 8 + 5 + 0 = 21.

> Een IPv6 adres bestaat uit 128 bits: de eerste 64 bits vormen het netwerkadres; de laatste 64 bits, het adres van de host in het netwerk.


## Communicatie met het internet: NAT

Een IP-pakket bestemd voor een apparaat buiten het lokale netwerk wordt ingepakt in een Ethernet- of WiFi-frame naar de gateway. De gateway stuurt dit pakket dan via de WAN-verbinding van het ISP-netwerk door naar de eerstvolgende router/gateway in het pad naar de bestemming.

De lokale router/gateway gebruikt hierbij vaak Network Address Translation (NAT). In het lokale netwerk worden "private" IP-adresssen gebruikt, bijvoorbeeld 192.168.4.23. Zo'n adres is onbruikbaar in het publieke internet. De gateway vervangt daarom het afzender-IP-adres van een uitgaand pakket door het eigen IP-adres in het ISP-netwerk. Dit is wel een publiek IP-adres.

Een gevolg hiervan is dat alle apparaten in het lokale netwerk hetzelfde publieke IP-adres krijgen, nl. het adres van de router/gateway.

### (+) Hoe werkt NAT?

Op een uitgaand pakket volgt vaak een antwoord van het apparaat elders in het internet. Hoe kan de gateway er dan voor zorgen dat dit antwoord bij het juiste apparaat in het lokale netwerk terecht komt? Immers, alle lokale apparaten delen hetzelfde publieke IP-adres.

```{margin}
Bekende poortnummers zijn bijvoorbeeld 80 (HTTP: webserver), 443 (HTTPS: webserver), 22 (SSH: secure shell).
```

De oplossing hiervoor is om deze NAT-vertaling niet (alleen) op het niveau van IP-pakketten te doen, maar ook op het niveau direct daarboven: van de UDP-datagrammen (berichten) en TCP-segmenten. Deze "hoger-niveau pakketten" bevatten een extra adres: het host-poortnummer. Het UDP- of TCP-protocol gebruikt dit poortnummer om aan te geven welke toepassing op de host aan deze communicatie gekoppeld is.

```{figure} images/udp-in-ip-pakket.png
:width: 600px

UDP-datagram ingepakt in een IP-pakket
```

Bij de NAT-vertaling wordt de lokale combinatie (IP-adres, poortnummer) omgezet in een publieke combinatie (gateway publiek IP-adres, poortnummer). Bij ontvangst van een inkomend bericht voert de gateway de omgekeerde omzetting uit.

| lokaal IP-adres | lokaal poortnr | publiek IP-adres | publiek poortnr |
| --:             |  --:           |   --:            | --:             |
| 192.168.4.23    | 8010           | 94.231.226.38    | 8113            |
| 192.168.4.37    | 8010           | 94.231.226.38    | 8117            |
| 192.168.4.23    | 3712           | 92.231.226.38    | 8114            |
| 192.168.4.30    | 3720           | 92.231.226.38    | 3512            |

Zoals je ziet delen alle lokale computers hetzelfde publieke IP-adres: het adres van de gateway. Alleen aan de hand van de (UDP, TCP, etc.) poortnummers kan de gateway het onderscheid maken, bij het ontvangen van een pakket "van buiten".

Deze tabel wordt ingevuld door de gateway: voor een uitgaand pakket met een nieuwe (IP-adres, poortnummer) combinatie wordt een nieuw publiek poortnummer toegewezen.

### (+) Van publiek internet naar lokaal netwerk

NAT-vertaling werkt op zich prima voor het lokale verkeer naar het internet, zoals bij het gebruik van (web)servers in het internet.
Maar op deze manier is een apparaat in het lokale netwerk niet van buiten te bereiken.
Dat kan een probleem zijn bijv. voor bepaalde games; of als je een lokale computer wilt gebruiken als web- of file-server in het publieke internet.

Een oplossing hiervoor is het *openzetten van een poort* (port sharing): je kunt voor de combinatie van een lokaal (IP-adres, poortnr) een publiek poortnr in deze tabel zetten.
Via deze publieke poort is je lokale apparaat dan van buiten bereikbaar.
Dit kan gevolgen hebben voor de veiligheid van het netwerk: als de besturing van een computer in het netwerk van buiten overgenomen kan worden, is het hele lokale netwerk toegankelijk.
 